## 허깅페이스에서 모델받아 다국어번역 서비스만들기
* https://huggingface.co/

In [2]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

ko_text = "이것은 m2m모델로 만든 다국어 번역기입니다."

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_1.2B")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_1.2B")

# 한국어를  영어로
tokenizer.src_lang = "ko"
encoded_hi = tokenizer(ko_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("en"))
result1 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result1)


# 한국어를 일본어로
tokenizer.src_lang = "ko"
encoded_zh = tokenizer(ko_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_zh, forced_bos_token_id=tokenizer.get_lang_id("ja"))
result2 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result2)

['This is a multilingual translator made with the m2m model.']
['これは、m2mモデルで作成された多言語翻訳機です。']


In [5]:
import gradio as gr
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# 모델과 토크나이저 로딩
model_name = "facebook/m2m100_1.2B"
model = M2M100ForConditionalGeneration.from_pretrained(model_name)
tokenizer = M2M100Tokenizer.from_pretrained(model_name)

# 지원 언어 목록
lang_dict = {
    "English": "en",
    "Korean": "ko",
    "Japanese": "ja",
    "Chinese (Simplified)": "zh",
    "French": "fr",
    "German": "de",
    "Spanish": "es",
    "Arabic": "ar",
    "Russian": "ru"
}

# 번역 함수
def translate(text, src_lang, tgt_lang):
    if not text.strip():
        return ""
    tokenizer.src_lang = lang_dict[src_lang]
    encoded = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded, forced_bos_token_id=tokenizer.get_lang_id(lang_dict[tgt_lang])
    )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

# Gradio UI 구성
with gr.Blocks() as demo:
    gr.Markdown("# 🌐 다국어 번역기 (M2M100 기반)")

    # 언어 선택: 같은 줄에 두 개 드롭다운
    with gr.Row():
        src_lang = gr.Dropdown(choices=list(lang_dict.keys()), value="Korean", label="원본 언어")
        tgt_lang = gr.Dropdown(choices=list(lang_dict.keys()), value="English", label="번역 언어")

    # 텍스트 입력/출력: 같은 줄에 두 개 텍스트박스
    with gr.Row():
        input_text = gr.Textbox(lines=10, label="원본 텍스트 입력")
        output_text = gr.Textbox(lines=10, label="번역 결과", interactive=False)

    # 버튼
    translate_btn = gr.Button("번역하기")
    translate_btn.click(fn=translate, inputs=[input_text, src_lang, tgt_lang], outputs=output_text)

# 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [6]:
demo.close()

Closing server running on port: 7862
